In [2]:
from datetime import datetime
from datetime import timezone
import polars as pl
import os
import plotly.express as px

DATA_DIRECTORY = os.environ.get("DATA_DIRECTORY")
df = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "processed", "pipeline",  "calibrated_1_h_acropolis.parquet"))
df_p = pl.read_parquet(os.path.join(DATA_DIRECTORY,"processed", "picarro", "Calibrated_1_h_DWD_Picarro_G2301_413.parquet"))

In [3]:
start_time = datetime(2024, 4, 1, 0, 0, 0).replace(tzinfo=timezone.utc)

# until when where the systems on the roof-top

dates = [
        (1, datetime(2024, 2, 29, 0, 0, 0).replace(tzinfo=timezone.utc)),
        (2, datetime(2024, 7, 30, 0, 0, 0).replace(tzinfo=timezone.utc)),
        (3, datetime(2024, 6, 27, 0, 0, 0).replace(tzinfo=timezone.utc)),
        (4, datetime(2024, 7, 30, 0, 0, 0).replace(tzinfo=timezone.utc)),
        (5, datetime(2024, 2, 28, 0, 0, 0).replace(tzinfo=timezone.utc)),
        (6, datetime(2024, 8, 30, 0, 0, 0).replace(tzinfo=timezone.utc)),
        (7, datetime(2024, 6, 23, 0, 0, 0).replace(tzinfo=timezone.utc)),
        (8, datetime(2024, 3, 15, 0, 0, 0).replace(tzinfo=timezone.utc)),
        (9, datetime(2024, 6, 26, 0, 0, 0).replace(tzinfo=timezone.utc)),
        (10, datetime(2024, 4, 11, 0, 0, 0).replace(tzinfo=timezone.utc)),
        (11, datetime(2024, 4, 11, 0, 0, 0).replace(tzinfo=timezone.utc)),
        (12, datetime(2024, 2, 14, 0, 0, 0).replace(tzinfo=timezone.utc)),
        (13, datetime(2024, 2, 22, 0, 0, 0).replace(tzinfo=timezone.utc)),
        (14, datetime(2024, 6, 23, 0, 0, 0).replace(tzinfo=timezone.utc)),
        (15, datetime(2024, 5, 20, 0, 0, 0).replace(tzinfo=timezone.utc)),
        (16, datetime(2024, 2, 8, 0, 0, 0).replace(tzinfo=timezone.utc)),
        #(17, datetime(2024, 7, 9, 0, 0, 0).replace(tzinfo=timezone.utc)),
        (18, datetime(2024, 2, 8, 0, 0, 0).replace(tzinfo=timezone.utc)),
        (20, datetime(2024, 2, 14, 0, 0, 0).replace(tzinfo=timezone.utc)),    
    ]

In [5]:
df_p

creation_timestamp,picarro_corrected,h2o_reported,std
"datetime[μs, UTC]",f64,f64,f64
2023-07-01 00:00:00 UTC,441.714682,2.292852,4.137609
2023-07-01 01:00:00 UTC,452.222705,2.309166,1.101797
2023-07-01 02:00:00 UTC,442.597324,2.229544,4.72904
2023-07-01 03:00:00 UTC,432.832156,2.10657,1.842449
2023-07-01 04:00:00 UTC,428.758025,2.063289,1.629
2023-07-01 05:00:00 UTC,432.761072,2.097293,2.260691
2023-07-01 06:00:00 UTC,439.11606,2.125937,0.681823
2023-07-01 07:00:00 UTC,436.138227,2.150116,2.081675
2023-07-01 08:00:00 UTC,428.606355,2.039214,2.252567


In [6]:
end_time = datetime(2024, 8, 1, 0, 0, 0).replace(tzinfo=timezone.utc)
df_p_filtered = df_p.filter(pl.col("creation_timestamp").is_between(start_time, end_time)) \
    .with_columns(sys_name_short = pl.lit("DWD_Picarro_G2301_413")) \
    .rename({"picarro_corrected": "co2_corrected"}) \
    .select("creation_timestamp","sys_name_short", "co2_corrected")


In [7]:
df.head(1).collect()

system_id,sys_name_short,creation_timestamp,system_name,gmp343_raw,gmp343_compensated,gmp343_filtered,gmp343_temperature,sht45_humidity,sht45_temperature,bme280_humidity,bme280_temperature,bme280_pressure,revision,receipt_timestamp,h2o_ah,h2o_v%,gmp343_dry,slope,intercept,wxt532_speed_avg,wxt532_speed_min,wxt532_speed_max,wxt532_direction_avg,wxt532_direction_min,wxt532_direction_max,wxt532_last_update_time,wxt532_temperature,wxt532_heating_voltage,wxt532_supply_voltage,wxt532_reference_voltage,enclosure_bme280_humidity,enclosure_bme280_pressure,enclosure_bme280_temperature,raspi_cpu_usage,raspi_cpu_temperature,raspi_disk_usage,raspi_memory_usage,ups_battery_error_detected,ups_battery_above_voltage_threshold,ups_battery_is_fully_charged,ups_powered_by_grid,slope_interpolated,intercept_interpolated,gmp343_corrected,date,std,gmp343_temperature_change
i64,str,"datetime[μs, UTC]",str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,"datetime[ns, UTC]",f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,date,f64,f64
1,"""acropolis-1""",2024-01-01 00:00:00 UTC,null,400.505302,463.088532,463.121857,29.937897,20.402088,26.196974,17.096427,26.242816,931.638821,25.0,2024-01-01 00:30:00.151141888 UTC,6.172328,0.926735,467.453903,null,null,4.443333,4.135,4.715,221.3,214.45,227.3,1.7041e9,null,null,null,null,13.303167,949.992333,32.521833,0.034317,51.811667,0.475,0.11445,0.0,1.0,1.0,1.0,null,null,null,2024-01-01,null,0.4


In [9]:
df_p_filtered.tail(1)

creation_timestamp,sys_name_short,co2_corrected
"datetime[μs, UTC]",str,f64
2024-08-01 00:00:00 UTC,"""DWD_Picarro_G2…",465.658036


In [10]:
# Utility
extracted_dates = [df_p_filtered]

for date in dates:

    df_temp = df.filter(pl.col("system_id") == date[0]) \
        .filter(pl.col("creation_timestamp").is_between(start_time, date[1])) \
        .rename({"gmp343_corrected": "co2_corrected"}) \
        .collect() \
    
    extracted_dates.append(df_temp)

df_extracted = pl.concat(extracted_dates, how="diagonal")

start_date = datetime(2024, 5, 16, 0, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 5, 21, 0, 0, 0).replace(tzinfo=timezone.utc)
df_extracted = df_extracted.filter(pl.col("creation_timestamp").is_between(start_date, end_date)) \

fig = px.line(df_extracted, x="creation_timestamp", y="gmp343_temperature", color="sys_name_short")
fig.update_layout(
    yaxis_title='Sensor Temperature (°C)',
    xaxis_title='',
    title='',
)
fig.show()
fig = px.line(df_extracted, x="creation_timestamp", y="enclosure_bme280_temperature", color="sys_name_short")
fig.update_layout(
    yaxis_title='Enclosure Temperature',
    xaxis_title='',
    title='',
)
fig.show()
fig = px.line(df_extracted, x="creation_timestamp", y="co2_corrected", color="sys_name_short")
fig.update_layout(
    yaxis_title='CO2 (ppm)',
    xaxis_title='',
    title='',
)
fig.show()

In [11]:
# Utility
extracted_dates = [df_p_filtered]

for date in dates:

    df_temp = df.filter(pl.col("system_id") == date[0]) \
        .filter(pl.col("creation_timestamp").is_between(start_time, date[1])) \
        .rename({"gmp343_corrected": "co2_corrected"}) \
        .collect() \
        .join(df_p.select("creation_timestamp", "picarro_corrected"), on="creation_timestamp") \
        .with_columns((pl.col("co2_corrected") - pl.col("picarro_corrected")).alias("diff"))
    
    extracted_dates.append(df_temp)

df_extracted = pl.concat(extracted_dates, how="diagonal")

start_date = datetime(2024, 5, 30, 0, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2024, 6, 13, 0, 0, 0).replace(tzinfo=timezone.utc)
df_extracted = df_extracted.filter(pl.col("creation_timestamp").is_between(start_date, end_date)) \

fig = px.line(df_extracted, x="creation_timestamp", y="gmp343_temperature", color="sys_name_short")
fig.show()
fig = px.line(df_extracted, x="creation_timestamp", y="enclosure_bme280_temperature", color="sys_name_short")
fig.show()
fig = px.line(df_extracted, x="creation_timestamp", y="co2_corrected", color="sys_name_short")
fig.show()
fig = px.line(df_extracted, x="creation_timestamp", y="diff", color="sys_name_short")
fig.show()



In [ ]:
df_extracted